In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable

torch.manual_seed(0)

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

batch_size = 1

data_size = [100, 300, 600, 900, 1200, 1500, 2300]

#normal_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CNN_dataset300\\normal'
#abnormal_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CNN_dataset300\\abnormal'

normal_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\normal'
abnormal_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\abnormal'

BEST_MODEL_PATH = 'CAE_best_model.pth'

In [3]:
normal_set = datasets.ImageFolder(
    normal_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

abnormal_set = datasets.ImageFolder(
    abnormal_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

In [4]:
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        
        # Encoder
        self.cnn_layer1 = nn.Sequential(
                        nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
                        nn.ReLU(),
                         nn.MaxPool2d(2,2))

        self.cnn_layer2 = nn.Sequential(
                                nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                 nn.MaxPool2d(2,2))
        
        self.cnn_layer3 = nn.Sequential(
                                nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                 nn.MaxPool2d(2,2))

        # Decoder        
        self.tran_cnn_layer1 = nn.Sequential(
                        nn.ConvTranspose2d(64, 32, kernel_size = 2, stride = 2, padding=0),
                        nn.ReLU())
        
        self.tran_cnn_layer2 = nn.Sequential(
                        nn.ConvTranspose2d(32, 16, kernel_size = 2, stride = 2, padding=0),
                        nn.ReLU())

        self.tran_cnn_layer3 = nn.Sequential(
                        nn.ConvTranspose2d(16, 3, kernel_size = 2, stride = 2, padding=0),
                        nn.Sigmoid())
            
            
    def forward(self, x):
        output = self.cnn_layer1(x)
        output = self.cnn_layer2(output)
        output = self.cnn_layer3(output)        
        output = self.tran_cnn_layer1(output)
        output = self.tran_cnn_layer2(output)
        output = self.tran_cnn_layer3(output)

        return output

model = ConvAutoEncoder()
model.cuda()

ConvAutoEncoder(
  (cnn_layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cnn_layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cnn_layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (tran_cnn_layer1): Sequential(
    (0): ConvTranspose2d(64, 32, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU()
  )
  (tran_cnn_layer2): Sequential(
    (0): ConvTranspose2d(32, 16, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU()
  )
  (tran_cnn_layer3): Sequential(
    (0): ConvTranspose2d(16, 3, kernel_size=(2, 2), stride=(2, 2))
    (1): Sigmoid()
  

In [5]:
model = ConvAutoEncoder()
model.cuda()
model.load_state_dict(torch.load(BEST_MODEL_PATH))
criterion = nn.MSELoss()

In [6]:
import math
    
def myEquation4(x):    
    normal_err = 0
    abnormal_err = 0    
    
    for i, (X_normal, _) in enumerate(normal_loader):
        X_normal = X_normal.cuda()
        output = model(X_normal)
        loss = criterion(X_normal, output)
        
        if loss.item() > x:
            normal_err += 1
        
    for i, (X_abnormal, _) in enumerate(abnormal_loader):
        X_abnormal = X_abnormal.cuda()
        output = model(X_abnormal)
        loss = criterion(X_abnormal, output)
        
        if loss.item() < x:
            abnormal_err += 1
      
    err = ((normal_err/len(normal_loader)) + (abnormal_err/len(abnormal_loader)))/2.0
    
    return err

gr = (math.sqrt(5) + 1) / 2

def gss(f, a, b, tol=1e-7):
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    while abs(b - a) > tol:
        if f(c) < f(d):
            b = d
        else:
            a = c

        # We recompute both c and d here to avoid loss of precision which may lead to incorrect results or infinite loop
        c = b - (b - a) / gr
        d = a + (b - a) / gr

    return (b + a) / 2

In [7]:
for number, dsize in enumerate(data_size, 0):
    normal_loader = torch.utils.data.DataLoader(normal_set, batch_size=batch_size, shuffle=True)
    abnormal_loader = torch.utils.data.DataLoader(list(abnormal_set)[:dsize], batch_size=batch_size,shuffle=True)
    
    threshold = gss(myEquation4, 0.0005, 0.0014)
    err = myEquation4(threshold)
    print(f"Number of abnormal data : {dsize}\nthreshold : {threshold}\nerr : {err}\n")
    
    with open(f'data/threshold.txt', 'a') as f:
        f.write(f'{threshold}\n')

Number of abnormal data : 100
threshold : 0.0013999518665103378
err : 0.0

Number of abnormal data : 300
threshold : 0.0012134022995547724
err : 0.0033333333333333335

Number of abnormal data : 600
threshold : 0.001127942110823426
err : 0.0

Number of abnormal data : 900
threshold : 0.001127942110823426
err : 0.0



KeyboardInterrupt: 